# CORD

In [4]:
import pandas as pd
import numpy as np
from pathlib import Path, PurePath
from cord.core import parallel, ifnone
import json
import pprint
from typing import List
from functools import reduce
import multiprocessing
from functools import partial
from nltk.corpus import stopwords
import nltk
from rank_bm25 import BM25Okapi

## CORD Library

In [5]:
from cord.cord19 import *
from cord.core import parallel, ifnone

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\dwight\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# 1. Reading the Research Papers

## From Paths

In [6]:
%time research_papers = ResearchPapers.from_data_dir()

Load JSON from data\CORD-19-research-challenge\2020-03-13\biorxiv_medrxiv\biorxiv_medrxiv


Load JSON from data\CORD-19-research-challenge\2020-03-13\comm_use_subset\comm_use_subset



Load JSON from data\CORD-19-research-challenge\2020-03-13\noncomm_use_subset\noncomm_use_subset


Load JSON from data\CORD-19-research-challenge\2020-03-13\pmc_custom_license\pmc_custom_license


Building a BM25 index
Wall time: 3min 10s


## From Pickle

In [4]:
%time research_papers = ResearchPapers.from_pickle()

Wall time: 23 s


## Search Bar

In [5]:
research_papers.searchbar('vaccine transmission')

interactive(children=(Text(value='vaccine transmission', description='SearchTerms'), Output()), _dom_classes=(…